## Instalar dependencias

In [14]:
%pip install -q mediapipe
%pip install opencv-python
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Descargar modelo

En Unix se puede correr el siguiente comando. En Windows (sin wget) toca descargar el modelo de la URL y poner el archivo en la carpeta `./modelos`

In [2]:
!wget -q -O ./modelos/efficientdet_lite0.tflite -q https://storage.googleapis.com/mediapipe-models/object_detector/efficientdet_lite0/int8/1/efficientdet_lite0.tflite

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import mediapipe as mp
from mediapipe.tasks.python import vision
from mediapipe.tasks.python import BaseOptions
import cv2
import json

opciones = vision.ObjectDetectorOptions(
    base_options=BaseOptions(model_asset_path='./modelos/efficientdet_lite0.tflite'),
    max_results=5,
    score_threshold=0.5,
    running_mode=vision.RunningMode.VIDEO)

modelo = vision.ObjectDetector.create_from_options(opciones)

nombre_archivo = 'GZZ_Leila_03_Carnaval_1 - pan.do_ra AI.webm';

video = cv2.VideoCapture('./estaticos/videos/' + nombre_archivo)
ancho_video = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
alto_video = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_fotogramas = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fps = video.get(cv2.CAP_PROP_FPS)
print(ancho_video, alto_video)

apariciones = []
print(fps, total_fotogramas)

for i in range(total_fotogramas):
    respuesta, fotograma = video.read()
    
    if respuesta == False:
        break
    
    fotograma_rgb = cv2.cvtColor(fotograma, cv2.COLOR_BGR2RGB)
    fotograma_rgb = mp.Image(image_format=mp.ImageFormat.SRGB, data=fotograma_rgb)
    milisegundos = int(1000 * i / fps)
    
    # Detectar objetos
    resultados = modelo.detect_for_video(fotograma_rgb, milisegundos)
    
    for prediccion in resultados.detections:
        cuadro = prediccion.bounding_box
        x, y, ancho, alto = cuadro.origin_x, cuadro.origin_y, cuadro.width, cuadro.height
        categoria = prediccion.categories[0]
        confianza = categoria.score * 100
        nombre = categoria.category_name
        apariciones.append({'categoria': nombre, 'x': x, 'y': y, 'ancho': ancho, 'alto': alto, 'confianza': confianza, 'tiempo': milisegundos})
        cv2.rectangle(fotograma, (x, y), (x + ancho, y - 30), (100, 255, 0), -1)
        cv2.rectangle(fotograma, (x, y), (x + ancho, y + alto), (100, 255, 0), 2)
        cv2.putText(fotograma, f"{nombre} {confianza:.2f}%", (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    cv2.imshow('Video', fotograma)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()

datos = json.dumps({'video': nombre_archivo, 'ancho': ancho_video, 'alto': alto_video, 'fps': fps, 'apariciones': apariciones}, separators=(',', ":"))

with open(nombre_archivo + ".json", "w") as outfile:
    outfile.write(datos)


640 480
25.0 5665


g:\dev\enflujo\enflujo-emporio-no-benevolente\.venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 